<a href="https://colab.research.google.com/github/leemichaelwaters/analyst-take-home-task/blob/master/LEE_WATERS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHOP Data Analyst: Data Exercise

## Part 1: Assemble the project cohort

The project goal is to identify patients seen for drug overdose, determine if they had an active opioid at the start of the encounter, and if they had any readmissions for drug overdose.

Your task is to assemble the study cohort by identifying encounters that meet the following criteria:

1. ~The patient’s visit is an encounter for drug overdose~
2. ~The hospital encounter occurs after July 15, 1999~
3. ~The patient’s age at time of encounter is between 18 and 35 (Patient is considered to be 35 until turning 36)~

In [ ]:
# Import packages
import pandas as pd
import numpy as np

from datetime import datetime
from google.colab import files

### 1. The patient's visit is for drug overdose

In [ ]:
# Import encounters
df_encounters = pd.read_csv('https://raw.githubusercontent.com/chop-analytics/analyst-take-home-task/master/datasets/encounters.csv')
df_drug_overdose = df_encounters[df_encounters['REASONDESCRIPTION'] == 'Drug overdose']

# Export CSV
def export_dataframe_to_csv(df, filename):
    # Export DataFrame to CSV
    df.to_csv(filename, index=False)

    # Download the CSV file
    files.download(filename)

#export_dataframe_to_csv(df_drug_overdose, 'df_drug_overdose.csv')

### 2. The visit occurs after July 15, 1999

In [ ]:
# Convert the 'START' column to datetime
df_start_datetime = df_drug_overdose.copy()
df_start_datetime['START'] = pd.to_datetime(df_start_datetime['START'])

# Filter for hospital encounters after July 15, 1999
df_after_1999_07_15 = df_start_datetime[df_start_datetime['START'] > '1999-07-15']

# Export CSV
#export_dataframe_to_csv(df_after_1999_07_15, 'df_after_1999_07_15.csv')

### The patient's age is between 18 and 35

In [ ]:
# Import patients
df_patients = pd.read_csv('https://raw.githubusercontent.com/chop-analytics/analyst-take-home-task/master/datasets/patients.csv')

# Clean data
df_clean = df_patients.rename(columns={'Id': 'PATIENT'})

# Merge dataframes
df_merge = pd.merge(df_after_1999_07_15, df_clean)

# Convert the 'BIRTHDATE' column to datetime
df_merge['BIRTHDATE'] = pd.to_datetime(df_merge['BIRTHDATE'])

# Create 'AGE' column
today = datetime.today().date()
df_merge['AGE'] = df_merge['BIRTHDATE'].apply(lambda x: today.year - x.year - ((today.month, today.day) < (x.month, x.day)))

# Filter for ages between 18 and 35
df_age_18_to_35 = df_merge[(df_merge['AGE'] >= 18) & (df_merge['AGE'] <= 35)]

# Export CSV
#export_dataframe_to_csv(df_age_18_to_35, 'df_age_18_to_35.csv')

## Part 2: Create additional fields

With your durg overdose encounter, create the following indicators:

1. ~`DEATH_AT_VISIT_IND`: `1` if patient died during the drug overdose encounter, `0` if the patient died at a different time~
2. ~`COUNT_CURRENT_MEDS`: Count of active medications at the start of the drug overdose encounter~
3. ~`CURRENT_OPIOID_IND`: `1` if the patient had at least one active medication at the start of the overdose encounter that is on the Opioids List (provided below), `0` if not~
4. ~`READMISSION_90_DAY_IND`: `1` if the visit resulted in a subsequent drug overdose readmission within 90 days, `0` if not~
5. ~`READMISSION_30_DAY_IND`: `1` if the visit resulted in a subsequent drug overdose readmission within 30 days, `0` if not~
6. ~`FIRST_READMISSION_DATE`: The date of the index visit's first readmission for drug overdose. Field should be left as `N/A` if no readmission for drug overdose within 90 days~


### Did patient die during visit?

In [ ]:
# Create a copy of the DataFrame
df_death_at_visit = df_age_18_to_35.copy()

# Convert 'STOP' column to datetime
df_death_at_visit['STOP'] = pd.to_datetime(df_death_at_visit['STOP'])

# Remove the time component from 'STOP' column and keep only the date
df_death_at_visit['STOP'] = df_death_at_visit['STOP'].dt.date

# Convert 'DEATHDATE' column to datetime
df_death_at_visit['DEATHDATE'] = pd.to_datetime(df_death_at_visit['DEATHDATE'])

# Create the 'DEATH_AT_VISIT_IND' column based on conditions
df_death_at_visit['DEATH_AT_VISIT_IND'] = 0
df_death_at_visit.loc[df_death_at_visit['STOP'] == df_death_at_visit['DEATHDATE'], 'DEATH_AT_VISIT_IND'] = 1

# Export CSV
#export_dataframe_to_csv(df_death_at_visit, 'df_death_at_visit.csv')

### Count of active medications during visit

In [ ]:
# Import medications
df_medications = pd.read_csv('https://raw.githubusercontent.com/chop-analytics/analyst-take-home-task/master/datasets/medications.csv')

# Convert columns to datetime
df_medications['START'] = pd.to_datetime(df_medications['START'])
df_medications['STOP'] = pd.to_datetime(df_medications['STOP'])

# Create a copy of df_death_at_visit and name it df_current_meds
df_current_meds = df_death_at_visit.copy()

# Count active medications at the start of drug overdose encounter
df_current_meds['COUNT_CURRENT_MEDS'] = 0
for index, row in df_current_meds.iterrows():
    patient = row['PATIENT']
    encounter_date = row['START']
    df_current_meds.loc[index, 'COUNT_CURRENT_MEDS'] = len(df_medications[
      (df_medications['PATIENT'] == patient) &
      (encounter_date > df_medications['START']) &
      ((encounter_date < df_medications['STOP']) | pd.isna(df_medications['STOP']))
      ])

# Export CSV
#export_dataframe_to_csv(df_current_meds, 'df_current_meds.csv')

### Was patient taking medicine in Opiod List?

In [ ]:
# Create a copy of df_death_at_visit and name it df_current_meds
df_current_opiod = df_current_meds.copy()

# Define opiods list
opiods = [
    'Hydromorphone 325 MG',
    'Fentanyl 100 MCG',
    'Oxycodone-acetaminophen 100ML',
]

# Index if the patient had at least one active medication at the start of the
# overdose encounter that is on the Opioids List
df_current_opiod['CURRENT_OPIOID_IND'] = 0
for index, row in df_current_opiod.iterrows():
    patient = row['PATIENT']
    encounter_date = row['START']
    filtered_medications = df_medications[
        (df_medications['PATIENT'] == patient) &
        (encounter_date > df_medications['START']) &
        ((encounter_date < df_medications['STOP']) | pd.isna(df_medications['STOP']))
    ]
    if not filtered_medications.empty:
        opioids_present = filtered_medications['DESCRIPTION'].isin(opiods)
        if opioids_present.any():
            df_current_opiod.loc[index, 'CURRENT_OPIOID_IND'] = 1

# Export CSV
#export_dataframe_to_csv(df_current_opiod, 'df_current_opiod.csv')

### Was patient readmitted within 90 days? If so, when?

In [ ]:
# Create a copy of the dataframe
df_readmission_90 = df_current_opiod.copy()

# Function to calculate the readmission indicator and first readmission date
def calculate_readmission_ind(df, days, readmission_col, first_readmission_col):
    # Sort the dataframe by patient ID and visit date
    df.sort_values(['PATIENT', 'START'], inplace=True)

    df[readmission_col] = 0
    df[first_readmission_col] = pd.NaT  # Set column to NaT

    # Iterate over each patient
    for patient_id, patient_group in df.groupby('PATIENT'):
        # Get the indices of the patient's visits
        indices = patient_group.index.tolist()

        # Iterate over the indices starting from the second visit
        for i in range(1, len(indices)):
            current_index = indices[i]
            previous_index = indices[i-1]
            current_start = df.at[current_index, 'START']
            previous_start = df.at[previous_index, 'START']

            # Check if the previous visit is within the specified days
            if (current_start - previous_start).days <= days:
                df.at[current_index, readmission_col] = 1
                if pd.isna(df.at[current_index, first_readmission_col]):  # Check if value is NaN
                    df.at[current_index, first_readmission_col] = df.at[previous_index, 'START']

    return df

# Apply the function to calculate READMISSION_90_DAY_IND with 90 days and FIRST_READMISSION_DATE
df_readmission_90 = calculate_readmission_ind(df_readmission_90, 90, 'READMISSION_90_DAY_IND', 'FIRST_READMISSION_DATE_90')

# Export CSV
#export_dataframe_to_csv(df_readmission_90, 'df_readmission_90.csv')

### Was patient readmitted within 30 days? If so, when?

---



In [ ]:
# Create a copy of the dataframe
df_readmission_30 = df_readmission_90.copy()

# Apply the function to calculate READMISSION_30_DAY_IND with 30 days
df_readmission_30 = calculate_readmission_ind(df_readmission_30, 30, 'READMISSION_30_DAY_IND', 'FIRST_READMISSION_DATE_30')

# Export CSV
#export_dataframe_to_csv(df_readmission_30, 'df_readmission_30.csv')

# Part 3: Export the data to a CSV file

Export a dataset containing these required fields:

| Field name              | Field Description                                                                                                  | Data Type |
|---------------------------|-------------------------------------------------------------------------------------------------------------------|-----------|
| `PATIENT_ID`              | Patient identifier                                                                                                | Character String |
| `ENCOUNTER_ID`            | Visit identifier                                                                                                  | Character string |
| `HOSPITAL_ENCOUNTER_DATE` | Beginning of hospital encounter date                                                                              | Date/time |
| `AGE_AT_VISIT`            | Patient age at admission                                                                                          | Num |
| `DEATH_AT_VISIT_IND`      | Indicator if the patient died during the drug overdose encounter. Leave N/A if patient has not died              | 0 /1 |
| `COUNT_CURRENT_MEDS`      | Count of active medications at the start of the drug overdose encounter                                           | Num |
| `CURRENT_OPIOID_IND`      | Indicator if the patient had at least one active medication at the start of the overdose encounter that is on the Opioids List (provided below) | 0/1 |
| `READMISSION_90_DAY_IND`  | Indicator if the visit resulted in a subsequent readmission within 90 days                                       | 0/1 |
| `READMISSION_30_DAY_IND`  | Indicator if the visit resulted in a subsequent readmission within 30 days                                       | 0/1 |
| `FIRST_READMISSION_DATE`  | Date of the first readmission for drug overdose within 90 days. Leave N/A if no readmissions for drug overdose within 90 days. | Date/time |




In [ ]:
# Create a copy of the dataframe
df_final = df_readmission_30.copy()

# Select columns
df_final = df_final[[
    'PATIENT',
    'Id',
    'START',
    'AGE',
    'DEATH_AT_VISIT_IND',
    'COUNT_CURRENT_MEDS',
    'CURRENT_OPIOID_IND',
    'READMISSION_90_DAY_IND',
    'READMISSION_30_DAY_IND',
    'FIRST_READMISSION_DATE_90',
]]

# Rename columns
df_final.rename(columns={
    'PATIENT': 'PATIENT_ID',
    'Id': 'ENCOUNTER_ID',
    'START': 'HOSPITAL_ENCOUNTER_DATE',
    'AGE': 'AGE_AT_VISIT',
    'FIRST_READMISSION_DATE_90': 'FIRST_READMISSION_DATE',
    }, inplace=True)

# Export CSV
export_dataframe_to_csv(df_final, 'LEE_WATERS.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>